# Chicago Taxi + Weather Data Integration & Modeling
**Purpose**: Combine taxi trips with weather data and create normalized data model  
**Input**: Raw taxi data + weather data from APIs  
**Output**: AWS-optimized dataset with master tables for efficient storage/querying

In [ ]:
import pandas as pd
import requests
import os
from datetime import datetime
from dateutil.relativedelta import relativedelta
import json

# Display settings for better data exploration
pd.set_option("display.max_columns", 30)

print("Chicago Taxi-Weather Integration Pipeline Started")

## Step 1: Load Taxi Data from Chicago Open Data API

In [ ]:
# Get historical date (2 months ago) for consistent data timeframe
target_date = datetime.now() - relativedelta(months=2)
date_str = target_date.strftime("%Y-%m-%d")

# Chicago Taxi Trips API configuration
DATASET_ID = "ajtu-isnz"
BASE_URL = f"https://data.cityofchicago.org/resource/{DATASET_ID}.json"

# Build query for full day of taxi trips
where_clause = (
    f"trip_start_timestamp >= '{date_str}T00:00:00' "
    f"AND trip_start_timestamp <= '{date_str}T23:59:59'"
)

params = {
    "$where": where_clause,
    "$limit": 30000,
}

headers = {
    "X-App-Token": os.environ.get("CHICAGO_API_TOKEN", "")
}

print(f"Fetching taxi data for: {date_str}")

In [ ]:
# Fetch taxi trips from API
response = requests.get(BASE_URL, headers=headers, params=params, timeout=60)
response.raise_for_status()
taxi_data = response.json()

# Convert to DataFrame
taxi_trips = pd.DataFrame(taxi_data)

print(f"Taxi records loaded: {len(taxi_trips):,}")
print(f"Columns: {len(taxi_trips.columns)}")
taxi_trips.head(3)

## Step 2: Data Cleaning & Preparation

In [ ]:
# Remove complex nested columns that cause join issues
columns_to_drop = [
    "pickup_centroid_location", 
    "dropoff_centroid_location",
    "pickup_census_tract", 
    "dropoff_census_tract"
]

taxi_trips.drop(columns_to_drop, axis=1, inplace=True, errors='ignore')
print(f"Dropped complex columns: {columns_to_drop}")

# Remove rows with missing critical data
before_cleanup = len(taxi_trips)
taxi_trips.dropna(inplace=True)
after_cleanup = len(taxi_trips)

print(f"Rows removed due to missing data: {before_cleanup - after_cleanup:,}")
print(f"Clean taxi records: {after_cleanup:,}")

In [ ]:
# Rename columns for clarity
taxi_trips.rename(columns={
    "pickup_community_area": "pickup_community_area_id",
    "dropoff_community_area": "dropoff_community_area_id"
}, inplace=True)

# Create weather correlation column (round trip start time to nearest hour)
taxi_trips["datetime_for_weather"] = pd.to_datetime(
    taxi_trips["trip_start_timestamp"]
).dt.floor("h")

print("Taxi data prepared for weather correlation")
print(f"Date range for weather join: {taxi_trips['datetime_for_weather'].min()} to {taxi_trips['datetime_for_weather'].max()}")

## Step 3: Load Weather Data from Open-Meteo API

In [ ]:
# Fetch weather data for the same date as taxi data
CHICAGO_LAT = 41.85
CHICAGO_LON = -87.65

weather_url = "https://api.open-meteo.com/v1/forecast"
weather_params = {
    "latitude": CHICAGO_LAT,
    "longitude": CHICAGO_LON,
    "start_date": date_str,
    "end_date": date_str,
    "hourly": "temperature_2m,wind_speed_10m,rain,precipitation"
}

weather_response = requests.get(weather_url, params=weather_params)
weather_response.raise_for_status()
weather_data = weather_response.json()

# Create weather DataFrame
weather_df = pd.DataFrame({
    "datetime": pd.to_datetime(weather_data["hourly"]["time"]),
    "temperature": weather_data["hourly"]["temperature_2m"],
    "wind_speed": weather_data["hourly"]["wind_speed_10m"],
    "rain": weather_data["hourly"]["rain"],
    "precipitation": weather_data["hourly"]["precipitation"]
})

print(f"Weather records loaded: {len(weather_df)}")
weather_df.head()

## Step 4: Join Taxi and Weather Data

In [ ]:
# Merge taxi trips with weather data based on hourly timestamps
taxi_with_weather = taxi_trips.merge(
    weather_df,
    left_on="datetime_for_weather",
    right_on="datetime",
    how="inner"  # Only keep trips with matching weather data
)

print(f"Successfully joined taxi-weather records: {len(taxi_with_weather):,}")
print(f"Join rate: {len(taxi_with_weather)/len(taxi_trips)*100:.1f}%")

# Show sample of joined data
taxi_with_weather.head(3)

## Step 5: Data Type Optimization for AWS Storage

In [ ]:
# Optimize data types to reduce memory usage and improve AWS performance
memory_before = taxi_with_weather.memory_usage(deep=True).sum()

# Define optimized data types
optimized_types = {
    "trip_end_timestamp": "datetime64[ns]",
    "trip_seconds": "int32",
    "trip_miles": "float32",
    "pickup_community_area_id": "int8",
    "dropoff_community_area_id": "int8",
    "fare": "float32",
    "tips": "float32",
    "tolls": "float32",
    "extras": "float32",
    "trip_total": "float32",
    "temperature": "float32",
    "wind_speed": "float32",
    "rain": "float32",
    "precipitation": "float32"
}

# Apply optimizations where columns exist
for col, dtype in optimized_types.items():
    if col in taxi_with_weather.columns:
        try:
            taxi_with_weather[col] = taxi_with_weather[col].astype(dtype)
        except Exception as e:
            print(f"Could not convert {col}: {e}")

memory_after = taxi_with_weather.memory_usage(deep=True).sum()
savings = (1 - memory_after/memory_before) * 100

print(f"Memory optimization: {savings:.1f}% reduction")
print(f"Before: {memory_before/1024/1024:.1f} MB")
print(f"After: {memory_after/1024/1024:.1f} MB")

## Step 6: Create Normalized Master Tables

In [ ]:
# Create Payment Type Master Table
payment_types = taxi_with_weather["payment_type"].drop_duplicates().reset_index(drop=True)
payment_type_master = pd.DataFrame({
    "payment_type_id": range(1, len(payment_types) + 1),
    "payment_type": payment_types
})

print(f"Payment types identified: {len(payment_type_master)}")
payment_type_master

In [ ]:
# Create Company Master Table
companies = taxi_with_weather["company"].drop_duplicates().reset_index(drop=True)
company_master = pd.DataFrame({
    "company_id": range(1, len(companies) + 1),
    "company": companies
})

print(f"Taxi companies identified: {len(company_master)}")
company_master.head(10)

## Step 7: Create Final Normalized Dataset

In [ ]:
# Join master tables to create normalized fact table
taxi_final = taxi_with_weather.merge(payment_type_master, on="payment_type")
taxi_final = taxi_final.merge(company_master, on="company")

# Remove original text columns (now replaced with IDs)
taxi_final.drop(["payment_type", "company"], axis=1, inplace=True)

print(f"Final normalized dataset: {len(taxi_final):,} records")
print(f"Columns: {len(taxi_final.columns)}")

# Calculate final memory savings
original_memory = taxi_with_weather.memory_usage(deep=True).sum()
final_memory = taxi_final.memory_usage(deep=True).sum()
total_savings = (1 - final_memory/original_memory) * 100

print(f"Total memory optimization: {total_savings:.1f}% reduction")
taxi_final.head(3)

## Step 8: Export Data for AWS Pipeline

In [ ]:
# Create output directory
output_dir = "csv/processed"
os.makedirs(output_dir, exist_ok=True)

# Export fact table
fact_table_path = f"{output_dir}/chicago_taxi_weather_fact.csv"
taxi_final.to_csv(fact_table_path, index=False)
print(f"Fact table saved: {fact_table_path}")

# Export master tables
payment_master_path = f"{output_dir}/payment_type_master.csv"
payment_type_master.to_csv(payment_master_path, index=False)
print(f"Payment master saved: {payment_master_path}")

company_master_path = f"{output_dir}/company_master.csv"
company_master.to_csv(company_master_path, index=False)
print(f"Company master saved: {company_master_path}")

print("\n=== CHICAGO TAXI-WEATHER PIPELINE COMPLETE ===")
print(f"Total records processed: {len(taxi_final):,}")
print(f"Memory optimized by: {total_savings:.1f}%")
print("Ready for AWS ingestion!")